In [3]:
!pip install faiss-cpu sentence-transformers openai pymupdf
!pip -q install rank-bm25

In [7]:
##import os
#os.environ["OPENAI_API_KEY"] = ""
#My Google collab

In [6]:
import os
from openai import OpenAI

api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Missing OPENAI_API_KEY environment variable. Please set it before running.")

client = OpenAI(api_key=api_key)

In [8]:
from google.colab import files
import os

# Upload-FSAE PDF and Q&A CSV file
uploaded = files.upload()

pdf_path = None
csv_file = None

for f in uploaded.keys():
    if f.endswith(".pdf"):
        pdf_path = f
    elif f.endswith(".csv"):
        csv_file = f

if not pdf_path or not csv_file:
    raise FileNotFoundError("Both a PDF and a CSV file are required.")

Saving FSAE_Rules_2024_V1.pdf to FSAE_Rules_2024_V1.pdf
Saving rule_retrieval_qa.csv to rule_retrieval_qa.csv


In [9]:
#import fitz
##import re

#doc = fitz.open(pdf_path)
#raw_text = "\n".join(page.get_text() for page in doc)

#def clean_text(text):
   # text = text.replace('\xa0', ' ')
    #text = re.sub(r'-\n', '', text)
    #text = re.sub(r'\n+', ' ', text)
    #text = re.sub(r'\s{2,}', ' ', text)
    #return text.strip()
#full_text = clean_text(raw_text)
import fitz
import re

# Force start at page 5 )
start_page = 4
doc = fitz.open(pdf_path)

def extract_text_from_page(p):
    return p.get_text("text")


HEADER_RES = [
    re.compile(r'^\s*Formula SAE.*Page\s+\d+\s+of\s+\d+\s*$', re.I),
    re.compile(r'^\s*Version\s+\d+(\.\d+)?\s+\d{1,2}\s+\w+\s+\d{4}\s*$', re.I),
    re.compile(r'^\s*\d+\s*$'),
]
TOC_LINE_RE = re.compile(r'.+\.\s?\.\s?\.\s+\d+$')
SECTION_BANNER_RE = re.compile(r'^[A-Z]{1,4}\s*-\s+.+$')

def clean_lines(lines):
    out = []
    for ln in lines:
        s = ln.rstrip().replace('\xa0', ' ')
        if any(rx.match(s) for rx in HEADER_RES):
            continue
        if TOC_LINE_RE.search(s):
            continue
        if SECTION_BANNER_RE.match(s):
            continue
        out.append(s)
    return out

pages = []
for i in range(start_page, len(doc)):
    t = extract_text_from_page(doc[i])
    lines = t.splitlines()
    lines = clean_lines(lines)
    pages.append("\n".join(lines))

raw_text = "\n".join(pages)
print("Collected text chars:", len(raw_text))

Collected text chars: 293737


In [10]:
import re

def clean_text(text):
    text = text.replace('\xa0', ' ')
    text = re.sub(r'-\n', '', text)
    text = re.sub(r'\n{3,}', '\n\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text.strip()

full_text = clean_text(raw_text)
print("Cleaned text chars:", len(full_text))

Cleaned text chars: 292886


In [11]:
import re, json
RULE_HEAD_RE = re.compile(r'(?m)^(?P<rid>[A-Z]{1,4}\.\d+(?:\.\d+)*)(?:[ \t]+(?P<title>.+))?$')

def parse_rules_from_text(text):
    lines = text.splitlines()
    rules = []
    cur_id, buf = None, []

    def flush():
        nonlocal cur_id, buf, rules
        if cur_id and buf:
            content = "\n".join(buf).strip()
            if content:
                if not content.startswith(cur_id):
                    content = f"{cur_id} " + content
                rules.append((cur_id, content))
        cur_id, buf = None, []

    for ln in lines:
        s = ln.strip()
        m = RULE_HEAD_RE.match(s)
        if m:
            flush()
            cur_id = m.group("rid").strip()
            buf = [s]
        else:
            if cur_id:
                buf.append(ln)

    flush()
    return rules

rule_pairs = parse_rules_from_text(full_text)
rule_chunks = {rid: txt for rid, txt in rule_pairs}

with open("rule_chunks.json", "w", encoding="utf-8") as f:
    json.dump(rule_chunks, f, indent=2, ensure_ascii=False)

print("Total rules parsed:", len(rule_chunks))
print("Sample keys:", list(sorted(rule_chunks.keys()))[:20])

Total rules parsed: 1758
Sample keys: ['AD.1', 'AD.1.1', 'AD.1.2', 'AD.1.3', 'AD.2', 'AD.2.1', 'AD.2.2', 'AD.2.2.1', 'AD.2.2.2', 'AD.2.2.3', 'AD.2.2.4', 'AD.2.3', 'AD.3', 'AD.3.1', 'AD.3.1.1', 'AD.3.1.2', 'AD.3.1.3', 'AD.3.1.4', 'AD.3.2', 'AD.3.3']


In [12]:
import pandas as pd
import re

rows = [{"rule_id": rid, "chars": len(txt), "text": txt} for rid, txt in rule_chunks.items()]
df_rules = pd.DataFrame(rows).sort_values(["rule_id"]).reset_index(drop=True)
df_rules.to_csv("rule_chunks_preview.csv", index=False)
print("Wrote: rule_chunks_preview.csv")

families = ["GR","AD","DR","V","F","T","EV","ES","IC","ICV"]
for fam in families:
    cnt = (df_rules["rule_id"].str.startswith(fam + ".")).sum()
    print(f"{fam}: {cnt} chunks")

Wrote: rule_chunks_preview.csv
GR: 91 chunks
AD: 73 chunks
DR: 36 chunks
V: 50 chunks
F: 292 chunks
T: 240 chunks
EV: 279 chunks
ES: 0 chunks
IC: 160 chunks
ICV: 0 chunks


In [13]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder
import numpy as np
import re
import faiss

rule_ids = list(rule_chunks.keys())
rule_texts = [rule_chunks[k] for k in rule_ids]

def tok(s):
    return re.findall(r"[A-Za-z0-9\.]+", s.lower())

bm25 = BM25Okapi([tok(t) for t in rule_texts])

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(rule_texts, normalize_embeddings=True, convert_to_numpy=True).astype("float32")

index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

ce_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

print(f"FAISS index built with {len(rule_texts)} rules.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

FAISS index built with 1758 rules.


In [14]:
import pandas as pd

df = pd.read_csv(csv_file)

if "ground_truth" in df.columns:
    df = df.rename(columns={"ground_truth": "answer"})

if "id" not in df.columns:
    df["id"] = [f"ret_{i+1:03}" for i in range(len(df))]

df = df[["id", "question", "answer"]]
qa_data = df.to_dict(orient="records")

print(f"Loaded {len(qa_data)} QA pairs.")

Loaded 1192 QA pairs.


In [15]:
def extract_rule_id(question):
    match = re.search(r'([A-Z]+\.\d+(?:\.\d+)*)', question)
    return match.group(1) if match else None

def retrieve_chunks(question, model, index, rule_ids, rule_texts, top_k=5):
    q_embedding = model.encode([question], normalize_embeddings=True).astype("float32")
    D, I = index.search(q_embedding, top_k)
    return [(rule_ids[i], rule_texts[i]) for i in I[0]]

def build_prompt(question, retrieved_texts):
    context = "\n".join(retrieved_texts)
    return f"""You are an expert in the FSAE competition rules.

Question:
{question}

Retrieved Rules:
{context}

Task:
- Respond ONLY with the exact rule text as shown in the retrieved rules.
- Do not paraphrase, summarize, reformat, or add any extra words.
- Keep punctuation, capitalization, and spacing exactly as in the retrieved rule text.
- If multiple rules are shown, return only the one that exactly matches the requested rule number.
- If no matching rule exists, respond with exactly: No applicable rule found.
"""

def get_gpt4_answer(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content.strip()

In [16]:
results = []
sampled_qa_data = qa_data

for idx, qa in enumerate(sampled_qa_data):
    try:
        question = qa["question"]
        qa_id = qa["id"]
        ground_truth = qa["answer"]

        rule_id = extract_rule_id(question)
        if rule_id and rule_id in rule_chunks:
            retrieved = [(rule_id, f"{rule_id} {rule_chunks[rule_id]}")]
        else:
            retrieved = retrieve_chunks(question, model, index, rule_ids, rule_texts, top_k=1)

        prompt = build_prompt(question, [text for _, text in retrieved])
        answer = get_gpt4_answer(prompt)

        results.append({
            "id": qa_id,
            "question": question,
            "ground_truth": ground_truth,
            "prediction": answer
        })

        print(f"{qa_id} ")

    except Exception as e:
        print(f"Error with {qa_id}: {e}")

ret_001 
ret_002 
ret_003 
ret_004 
ret_005 
ret_006 
ret_007 
ret_008 
ret_009 
ret_010 
ret_011 
ret_012 
ret_013 
ret_014 
ret_015 
ret_016 
ret_017 
ret_018 
ret_019 
ret_020 
ret_021 
ret_022 
ret_023 
ret_024 
ret_025 
ret_026 
ret_027 
ret_028 
ret_029 
ret_030 
ret_031 
ret_032 
ret_033 
ret_034 
ret_035 
ret_036 
ret_037 
ret_038 
ret_039 
ret_040 
ret_041 
ret_042 
ret_043 
ret_044 
ret_045 
ret_046 
ret_047 
ret_048 
ret_049 
ret_050 
ret_051 
ret_052 
ret_053 
ret_054 
ret_055 
ret_056 
ret_057 
ret_058 
ret_059 
ret_060 
ret_061 
ret_062 
ret_063 
ret_064 
ret_065 
ret_066 
ret_067 
ret_068 
ret_069 
ret_070 
ret_071 
ret_072 
ret_073 
ret_074 
ret_075 
ret_076 
ret_077 
ret_078 
ret_079 
ret_080 
ret_081 
ret_082 
ret_083 
ret_084 
ret_085 
ret_086 
ret_087 
ret_088 
ret_089 
ret_090 
ret_091 
ret_092 
ret_093 
ret_094 
ret_095 
ret_096 
ret_097 
ret_098 
ret_099 
ret_100 
ret_101 
ret_102 
ret_103 
ret_104 
ret_105 
ret_106 
ret_107 
ret_108 
ret_109 
ret_110 
ret_111 
r

In [18]:
import os, pandas as pd, importlib.util

!rm -rf design_qa
!git clone -q https://github.com/anniedoris/design_qa.git
!pip -q install rouge nltk

repo_dir = os.path.abspath("design_qa")
metrics_path = os.path.join(repo_dir, "eval", "metrics", "metrics.py")
spec = importlib.util.spec_from_file_location("dq_metrics", metrics_path)
dq_metrics = importlib.util.module_from_spec(spec)
spec.loader.exec_module(dq_metrics)
normalize_answer = dq_metrics.normalize_answer


df_results = pd.DataFrame(results)
df_results["ground_truth"] = df_results["ground_truth"].apply(lambda x: normalize_answer(str(x)))
df_results["prediction"] = df_results["prediction"].apply(lambda x: normalize_answer(str(x)))


eval_csv = "retrieval_evaluation.csv"
(df_results.rename(columns={"prediction": "model_prediction"})
          .loc[:, ["ground_truth", "model_prediction"]]
).to_csv(eval_csv, index=False)
print(f"{eval_csv} (columns: ground_truth, model_prediction)")

#scoring
overall_f1, per_f1 = dq_metrics.eval_retrieval_qa(eval_csv)
print(f"Official Retrieval F1: {overall_f1:.6f} on {len(per_f1)} questions")

# results.txt ---
with open("results.txt", "w", encoding="utf-8") as f:
    f.write("DesignQA Results\n")
    f.write("Subset: Retrieval\n")
    f.write(f"Num Questions: {len(per_f1)}\n")
    f.write(f"F1: {overall_f1:.6f}\n")
print("results.txt")

# F1
def bow_f1(pred, gt):
    pt = dq_metrics.normalize_answer(str(pred)).split()
    gt = dq_metrics.normalize_answer(str(gt)).split()
    return dq_metrics.token_f1_score(pt, gt)

df_detailed = (df_results
               .rename(columns={"prediction": "model_prediction"})
               .loc[:, ["id", "question", "ground_truth", "model_prediction"]]
              ).copy()
df_detailed["f1"] = df_detailed.apply(lambda r: bow_f1(r["model_prediction"], r["ground_truth"]), axis=1)
df_detailed.to_csv("retrieval_detailed_with_f1.csv", index=False)
print("retrieval_detailed_with_f1.csv (columns:id, question, ground_truth, model_prediction, f1)")

retrieval_evaluation.csv (columns: ground_truth, model_prediction)
Official Retrieval F1: 0.931206 on 1192 questions
results.txt
retrieval_detailed_with_f1.csv (columns:id, question, ground_truth, model_prediction, f1)
